### Challenge 20: Break fixed-nonce CTR statistically

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

<div class="alert alert-block alert-info">

In [this file](challenge-data/20.txt) find a similar set of Base64'd plaintext. Do with them exactly what you did with the first, but solve the problem differently.

Instead of making spot guesses at to known plaintext, treat the collection of ciphertexts the same way you would repeating-key XOR.

Obviously, CTR encryption appears different from repeated-key XOR, but with a fixed nonce they are effectively the same thing.

To exploit this: take your collection of ciphertexts and truncate them to a common length (the length of the smallest ciphertext will work).

Solve the resulting concatenation of ciphertexts as if for repeating-key XOR, with a key size of the length of the ciphertext you XOR'd.

</div>

In [33]:
import cryptopals as cp
import base64 as b64
from numpy.random import randint

Just like last exercise -- generate a random key

In [37]:
key = bytes(list(randint(0, 256, 16)))
nonce = [0]*8

Read in the challenge data file

In [56]:
f = open('./challenge-data/20.txt', 'r')
s_list = f.readlines()
f.close()

N_msgs = len(s_list)

<div class="alert alert-block alert-info">
    
In successive encryptions (not in one big running CTR stream), encrypt each line of the base64 decodes of the following, producing multiple independent ciphertexts:
    
</div>

In [70]:
ciphertexts = []

# Encrypt each message with the same (random, unknown to us) key
for msg in s_list:
    ciphertexts.append(cp.AESEncrypt(b64.b64decode(msg.strip('\n')), key, 'CTR', nonce))


In [86]:
english_chars = b'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ .,'

# First try...do it same as I did on 19.  Only decode first N_min bytes where 
# N_min is the shortest ciphertext.
N_min = len(min(ciphertexts, key=len))

# Setup array to keep track of the "score" for each key candidate, for each column.  
key_scores = np.zeros((N_min,256))

# Rank key guesses for all possible keys for each individual column of the ciphertext
for text_idx in range(len(ciphertexts)):  
    for byte_idx in range(N_min):      
        if len(ciphertexts[text_idx]) > byte_idx:              
            for key_byte_guess in range(256):          
                # Score key guesses based on english character count if 
                # all of the ciphertexts are decrypted with that guess.
                if (ciphertexts[text_idx][byte_idx] ^ key_byte_guess) in english_chars:              
                    key_scores[byte_idx, key_byte_guess] += 1

Find the highest ranked key for each column (based on its "English-ness") and generate the key-stream based on that

In [87]:
key_stream = bytearray(N_min)              
for ii in range(N_min):
    key_stream[ii] = key_scores[ii,:].argmax()

Now decrypt each row / ciphertext using our recovered key-stream.  We didn't need to find the key...just the key-stream it produced!

In [88]:
for ct in ciphertexts:
    for ii in range(N_min):
        pt_byte = key_stream[ii] ^ ct[ii]
        print(f'{chr(pt_byte)}', end='')
    print()
        

i'm rated "R"...this is a warning, ya better void / P
cuz I came back to attack others in spite- / Strike l
but don't be afraid in the dark, in a park / Not a sc
ya tremble like a alcoholic, muscles tighten up / Wha
suddenly you feel like your in a horror flick / You g
music's the clue, when I come your warned / Apocalyps
haven't you ever heard of a MC-murderer? / This is th
death wish, so come on, step to this / Hysterical ide
friday the thirteenth, walking down Elm Street / You 
this is off limits, so your visions are blurry / All 
terror in the styles, never error-files / Indeed I'm 
for those that oppose to be level or next to this / I
worse than a nightmare, you don't have to sleep a win
flashbacks interfere, ya start to hear: / The R-A-K-I
then the beat is hysterical / That makes Eric go get 
soon the lyrical format is superior / Faces of death 
mC's decaying, cuz they never stayed / The scene of a
the fiend of a rhyme on the mic that you know / It's 
melodies-unmakable, pattern-

---
Let's do the same thing, but instead of stopping at the length of the smallest ciphertext, we'll keep going and recover as much of the ciphertexts as we can. We could do better, but you get the point!

In [89]:
print('\n**********************************\n\n')
max_msg_len = 0
for msg in s_list:
    
    # print(b64.b64decode(msg))
    ciphertexts.append(cp.AESEncrypt(b64.b64decode(msg.strip('\n')), key, 'CTR', nonce))
    if len(ciphertexts[-1]) > max_msg_len:
        max_msg_len = len(ciphertexts[-1])
    
key_scores = np.zeros((max_msg_len,256))

for text_idx in range(len(ciphertexts)):  
    for byte_idx in range(max_msg_len):      
        if len(ciphertexts[text_idx]) > byte_idx:              
            for key_byte_guess in range(256):          
                # Score key guesses based on english character count if 
                # all of the ciphertexts are decrypted with that guess.
                if (ciphertexts[text_idx][byte_idx] ^ key_byte_guess) in english_chars:              
                    key_scores[byte_idx, key_byte_guess] += 1
  
key_stream = bytearray(max_msg_len)              
for ii in range(max_msg_len):
    key_stream[ii] = key_scores[ii,:].argmax()       
                
for ct in ciphertexts:
    for ii in range(max_msg_len):
        if ii < len(ct):
            pt_byte = key_stream[ii] ^ ct[ii]
            print(f'{chr(pt_byte)}', end='')
    print()
        
        
                


**********************************


i'm rated "R"...this is a warning, ya better void / Poets are paranoid, DJ's D-stroyed
cuz I came back to attack others in spite- / Strike like lightnin', It's quite frightenin'!
but don't be afraid in the dark, in a park / Not a scream or a cry, or a bark, more like a spark6
ya tremble like a alcoholic, muscles tighten up / What's that, lighten up! You see a sight but
suddenly you feel like your in a horror flick / You grab your heart then wish for tomorrow quick,
music's the clue, when I come your warned / Apocalypse Now, when I'm done, ya gone!
haven't you ever heard of a MC-murderer? / This is the death penalty,and I'm servin' a
death wish, so come on, step to this / Hysterical idea for a lyrical professionist!
friday the thirteenth, walking down Elm Street / You come in my realm ya get beat!
this is off limits, so your visions are blurry / All ya see is the meters at a volume
terror in the styles, never error-files / Indeed I'm known-your exil

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)